In [1]:
import pandas as pd
import psycopg2
import dotenv
import os

dotenv.load_dotenv()

PG_PASSWORD = os.getenv("PG_PASSWORD")
PG_USER = os.getenv("PG_USER")
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_DATABASE = os.getenv("PG_DB")

# Database connection parameters
import psycopg2
import os

conn = psycopg2.connect(
    host=PG_HOST,
    database=PG_DATABASE,
    user=PG_USER,
    password=PG_PASSWORD,
    port=PG_PORT,
    sslmode="require",
)

# Query to fetch tracking data
query = """
SELECT pt.frame_id, pt.timestamp, pt.player_id, pt.x, pt.y, p.jersey_number, p.player_name, p.team_id
FROM player_tracking pt
JOIN players p ON pt.player_id = p.player_id
JOIN teams t ON p.team_id = t.team_id
WHERE pt.game_id = '5uts2s7fl98clqz8uymaazehg';
"""
tracking_df = pd.read_sql_query(query, conn)
tracking_df.head()

C:\Users\sraul\AppData\Local\Temp\ipykernel_588\435546501.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tracking_df = pd.read_sql_query(query, conn)


,frame_id,timestamp,player_id,x,y,jersey_number,player_name,team_id
0,1722798989000,00:01:29,ball,40.733621,4.543208,0,Ball,1oyb7oym5nwzny8vxf03szd2h
1,1722798989000,00:01:29,2hr7skflmxmpiiauz04h185ey,36.468000,52.523279,77,B. Nsimba,4dtif7outbuivua8umbwegoo5
2,1722798989000,00:01:29,8lqo2ajhgjockasmd198nai1,68.297526,18.072211,3,J. Pupe,4dtif7outbuivua8umbwegoo5
3,1722798989000,00:01:29,39307g4mv5kms5rtlss0x756,35.632457,36.453173,98,J. Soladio,4dtif7outbuivua8umbwegoo5
4,1722798989000,00:01:29,8fs5klbsyx06c5ds9s1i3xu96,50.640328,24.364173,7,Hong Hyun-Seok,8y3iucyxguipljcmf87a11bk9


In [3]:
def execute_query(query):
    return pd.read_sql_query(query, conn)

In [4]:
pressing_query = """
SELECT 
    match_id,
    team_id,
    COUNT(*) AS total_press_attempts,
    SUM(CASE WHEN success = 1 THEN 1 ELSE 0 END) AS successful_presses,
    ROUND(SUM(CASE WHEN success = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS success_rate
FROM matchevents
WHERE 
    ball_state = 'InPlay' 
    AND ball_owning_team != team_id
    AND team_id IS NOT NULL
GROUP BY match_id, team_id
"""

pressing_df = execute_query(pressing_query)
print("\nPressing Success Rates:")
print(pressing_df)

C:\Users\sraul\AppData\Local\Temp\ipykernel_588\3273044507.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


DatabaseError: Execution failed on sql '
SELECT 
    match_id,
    team_id,
    COUNT(*) AS total_press_attempts,
    SUM(CASE WHEN success = 1 THEN 1 ELSE 0 END) AS successful_presses,
    ROUND(SUM(CASE WHEN success = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS success_rate
FROM matchevents
WHERE 
    ball_state = 'InPlay' 
    AND ball_owning_team != team_id
    AND team_id IS NOT NULL
GROUP BY match_id, team_id
': operator does not exist: boolean = integer
LINE 6:     SUM(CASE WHEN success = 1 THEN 1 ELSE 0 END) AS successf...
                                  ^
HINT:  No operator matches the given name and argument types. You might need to add explicit type casts.
